In [1]:
import requests
from web3 import Web3
import pandas as pd

import sys
sys.path.append("./chainklik")

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import config.config as cfg

In [9]:
w3 = Web3(Web3.HTTPProvider(cfg.config["eth_mainnet_http_url"]))

In [10]:
utils.get_function_hex("balanceOf(address)") == "0x70a08231"

True

In [11]:
utils.get_log_topic("Transfer(address,address,uint256)") == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"

True

In [15]:
val = payload.erc20_decimals_payload(0, "0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2")
resp = requests.post(cfg.config["eth_mainnet_http_url"], json=val)
dec = int(resp.json()['result'],16)
assert dec == 18

In [16]:
val

{'id': 0,
 'jsonrpc': '2.0',
 'method': 'eth_call',
 'params': [{'to': '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2',
   'data': '0x313ce567'}]}

In [17]:
val = payload.eth_block_payload(0, hex(18471066))
resp = requests.post(cfg.config["eth_mainnet_http_url"], json=val)
timestamp = resp.json()['result']['timestamp']
assert timestamp == "0x65412017"

In [18]:
wallet = "0x756F45E3FA69347A9A973A725E3C98bC4db0b5a0"
block_number = 0
val = payload.eth_balance_payload(0, wallet, hex(block_number))
resp = requests.post(cfg.config["eth_mainnet_http_url"], json=val)
assert int(resp.json()['result'],16)/1e18 == 200

In [19]:
val = payload.eth_logs_by_block_payload(0, hex(18471066), hex(18471066))
resp = requests.post(cfg.config["eth_mainnet_http_url"], json=val)
assert len(resp.json()['result']) == 580

In [20]:
# log is identified by address and topics
df_logs = pd.DataFrame(resp.json()['result'])
df_logs["timestamp"] = timestamp
log_sig = "Transfer(index_topic_1 address from, index_topic_2 address to, uint256 value)"
log_hash, log_name, log_topics, log_data = utils.parse_log_signature(log_sig)
df_logs = utils.parse_log_data(df_logs.loc[df_logs["topics"].str[0] == log_hash].copy(), log_topics, log_data, log_name = log_name)
assert df_logs.shape[0] == 385

In [21]:
val = payload.eth_logs_by_block_address_payload(0, hex(18470000), hex(18471066), "0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599")
resp = requests.post(cfg.config["eth_mainnet_http_url"], json=val)
assert len(resp.json()['result']) == 789

In [22]:
# test eth transactions

In [23]:
val

{'id': 0,
 'jsonrpc': '2.0',
 'method': 'eth_getLogs',
 'params': [{'fromBlock': '0x119d470',
   'toBlock': '0x119d89a',
   'address': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'}]}

In [ ]:
{'id': 0,
 'jsonrpc': '2.0',
 'method': 'eth_sendTransaction',
 'params': [{"from": "0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7",
      "to": "0xd46e8dd67c5d32be8058bb8eb970870f07244567",
      "gas": "0x76c0", 
      "gasPrice": "0x9184e72a000", 
      "value": "0x9184e72a",
    }]
}